In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_federated as tff

import collections

from absl import app

import nest_asyncio
nest_asyncio.apply()

from sklearn.model_selection import train_test_split

import sys
sys.path.append("/home/tester/Desktop/TF/federated/tensorflow_federated/examples/simple_fedavg")
print(sys.path)
import simple_fedavg_tff

2022-06-08 12:01:20.943231: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 12:01:20.943258: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


['/home/tester/Desktop/TF/TFF', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg']
['/home/tester/Desktop/TF/TFF', '/usr/lib/python39.zip', '/usr/lib/python3.9', '/usr/lib/python3.9/lib-dynload', '', '/home/tester/venv/lib/python3.9/site-packages', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg', '/home/tester/Desktop/federated/tensorflow_federated/examples/simple_fedavg']


In [2]:
# Training hyperparameters
TOTAL_ROUNDS = 50 #NUMBER OF TOTAL TRAINING ROUNDS
ROUNDS_PER_EVAL = 1 #HOW OFTEN TO EVALUATE
TRAIN_CLIENTS_PER_ROUND = 2 #HOW MANY CLIENTS TO SAMPLE PER ROUND
CLIENT_EPOCHS_PER_ROUND = 1 #NUMBER OF EPOCHS IN THE CLIENT TO TAKE PER ROUND
BATCH_SIZE = 16 #BATCH SIZE USED ON THE CLIENT
TEST_BATCH_SIZE = 128 #MINIBATCH SIZE OF TEST DATA
SERVER_LEARNING_RATE = 1.0 #SERVER LEARNING RATE
CLIENT_LEARNING_RATE = 0.1 #CLIENT LEARNING RATE
NUM_CLIENTS = 50

In [3]:
df = pd.read_csv("../datasets/TON_IoT-Datasets/Train_Test_datasets/Train_Test_Network_dataset/Train_Test_Network.csv")
df.head()
reduced_df = df[["src_port", "dst_port", "src_bytes", "dst_bytes", "missed_bytes" ,"label"]]

df = reduced_df

train, test = train_test_split(df, test_size=0.2)

train.head()

,src_port,dst_port,src_bytes,dst_bytes,missed_bytes,label
364743,36988,80,0,0,0,1
63151,42100,7878,0,0,0,0
4667,46912,15600,0,0,0,0
67693,42104,15600,0,0,0,0
408076,53136,9197,132,1770,0,0


In [4]:
# Percentage malware
perc = len(df.loc[df['label']==1])/len(df)
print(perc)

# Balance classes??

0.34930147513355586


In [5]:

#IID or NOIID? SPLIT DATA AMONG CLIENTS
client_id = np.random.choice(range(NUM_CLIENTS), size=len(train), replace=True)
#df['client_id'] = client_id

In [6]:
#TRAIN: Create a dict where keys are client ids format for tff.simulation.datasets.TestClientData
train_cl_dict = {}
for id in range(NUM_CLIENTS):
    tmp_train_df = train.loc[client_id == id]
    tmp_train_dict = {name: np.array(value) 
                         for name, value in tmp_train_df.items()}
    train_cl_dict[str(id)]=tmp_train_dict.copy()

In [7]:
#Determine max_client_ds_size
max_client_ds_size = -1
for id in range(NUM_CLIENTS):
    tmp_len = len(train.loc[client_id == id])
    if(tmp_len > max_client_ds_size):
        max_client_ds_size = tmp_len

In [8]:
# TEST
tmp_test_dict = {name: np.array(value)
    for name, value in test.items()}
test_cl_dict = {'0': tmp_test_dict}
test_ds_size = len(test)

In [9]:
#CONVERT TO TFF DATASET
#netw_ds = tf.data.Dataset.from_tensor_slices((netw_features_dict, netw_labels))
#netw_ds = tf.data.Dataset.from_tensor_slices(netw_features_dict)
train_netw_fd_ds = tff.simulation.datasets.TestClientData(train_cl_dict)
test_netw_fd_ds = tff.simulation.datasets.TestClientData(test_cl_dict)




2022-06-08 12:01:24.130199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-06-08 12:01:24.130229: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-06-08 12:01:24.130245: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (debian): /proc/driver/nvidia/version does not exist
2022-06-08 12:01:24.130458: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Test FedAVG

In [10]:
def evaluate(keras_model, test_dataset):
  """Evaluate the acurracy of a keras model on a test dataset."""
  metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
  for batch in test_dataset:
    #DONT KNOW WHAT DOES
    predictions = keras_model(batch['x'])
    metric.update_state(y_true=batch['y'], y_pred=predictions)
  return metric.result()


def get_custom_dataset():
  def element_fn(element):
    features = tf.convert_to_tensor([element['src_port'],element['dst_port'],element['src_bytes'],element['dst_bytes'],element['missed_bytes']], dtype=tf.int64)
    return collections.OrderedDict(
      # tf.expand_dims? ADD MORE COLUMNS
        x=features, y=element['label'])

  def preprocess_train_dataset(dataset):
    # Use buffer_size same as the maximum client dataset size,
    return dataset.map(element_fn).shuffle(buffer_size=max_client_ds_size).repeat(
        count=CLIENT_EPOCHS_PER_ROUND).batch(
            BATCH_SIZE, drop_remainder=False)
  def preprocess_test_dataset(dataset):
    return dataset.map(element_fn).batch(
        TEST_BATCH_SIZE, drop_remainder=False)
  netw_train = train_netw_fd_ds.preprocess(preprocess_train_dataset)
  netw_test = preprocess_test_dataset(
      test_netw_fd_ds.create_tf_dataset_from_all_clients())
  return netw_train, netw_test


def create_fedavg_model(only_digits=True):
  """The CNN model used in https://arxiv.org/abs/1602.05629.

  Args:
    only_digits: If True, uses a final layer with 10 outputs, for use with the
      digits only EMNIST dataset. If False, uses 62 outputs for the larger
      dataset.

  Returns:
    An uncompiled `tf.keras.Model`.
  """
  initializer = tf.keras.initializers.GlorotNormal(seed=0)
  return tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(5,)),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(8),
    tf.keras.layers.Dense(8),
    #tf.keras.layers.Dense(10, kernel_initializer=initializer),
    tf.keras.layers.Softmax()
    ])


def server_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=SERVER_LEARNING_RATE)


def client_optimizer_fn():
  return tf.keras.optimizers.SGD(learning_rate=CLIENT_LEARNING_RATE)

In [11]:
def main(argv):
  if len(argv) > 1:
    raise app.UsageError('Too many command-line arguments.')
  # If GPU is provided, TFF will by default use the first GPU like TF. The
  # following lines will configure TFF to use multi-GPUs and distribute client
  # computation on the GPUs. Note that we put server computatoin on CPU to avoid
  # potential out of memory issue when a large number of clients is sampled per
  # round. The client devices below can be an empty list when no GPU could be
  # detected by TF.
  client_devices = tf.config.list_logical_devices('GPU')
  server_device = tf.config.list_logical_devices('CPU')[0]
  tff.backends.native.set_local_python_execution_context(
      server_tf_device=server_device, client_tf_devices=client_devices)
  train_data, test_data = get_custom_dataset()

  def tff_model_fn():
    """Constructs a fully initialized model for use in federated averaging."""
    keras_model = create_fedavg_model(only_digits=True)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy()]
    return tff.learning.from_keras_model(
        keras_model,
        loss=loss,
        metrics=metrics,
        input_spec=train_data.element_type_structure)

  iterative_process = simple_fedavg_tff.build_federated_averaging_process(
      tff_model_fn, server_optimizer_fn, client_optimizer_fn)
  server_state = iterative_process.initialize()
  # Keras model that represents the global model we'll evaluate test data on.
  keras_model = create_fedavg_model(only_digits=True)
  for round_num in range(TOTAL_ROUNDS):
    sampled_clients = np.random.choice(
        train_data.client_ids,
        size=TRAIN_CLIENTS_PER_ROUND,
        replace=False)
    sampled_train_data = [
        train_data.create_tf_dataset_for_client(client)
        for client in sampled_clients
    ]
    server_state, train_metrics = iterative_process.next(
        server_state, sampled_train_data)
    print(f'Round {round_num}')
    print(f'\tTraining metrics: {train_metrics}')
    if round_num % ROUNDS_PER_EVAL == 0:
      server_state.model.assign_weights_to(keras_model)
      accuracy = evaluate(keras_model, test_data)
      print(f'\tValidation accuracy: {accuracy * 100.0:.2f}%')

In [12]:
# import nest_asyncio
# nest_asyncio.apply()

try:
    app.run(main([None]))
except:
    print("exec end error")

Round 0
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.62819636), ('loss', 1.645773), ('num_examples', 14626), ('num_batches', 915)])
	Validation accuracy: 64.79%
Round 1
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.6551795), ('loss', 1.6188245), ('num_examples', 14625), ('num_batches', 915)])
	Validation accuracy: 64.79%
Round 2
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.65780395), ('loss', 1.6161996), ('num_examples', 14781), ('num_batches', 925)])
	Validation accuracy: 64.79%
Round 3
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.6503034), ('loss', 1.6237006), ('num_examples', 14667), ('num_batches', 918)])
	Validation accuracy: 64.79%
Round 4
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.64958423), ('loss', 1.6244196), ('num_examples', 14791), ('num_batches', 925)])
	Validation accuracy: 64.79%
Round 5
	Training metrics: OrderedDict([('sparse_categorical_accuracy', 0.6505416), ('lo

FATAL Flags parsing error: Unknown command line flag 'ip'
Pass --helpshort or --helpfull to see help on flags.
